In [ ]:
%cd ../
%pip install -e .
%cd tutorials

In [ ]:
import json

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch

### 1. Import and runway from the LARD dataset

In [ ]:
img = mpimg.imread("LARD_example.jpg")
plt.imshow(img)

df = pd.read_csv("extract_labeling_Bing.csv", delimiter=";")
dic = df.to_dict(orient="records")[0]

TL = [dic["x_TL"], dic["y_TL"]]
plt.scatter(TL[0], TL[1], label="Top Left")

TR = [dic["x_TR"], dic["y_TR"]]
plt.scatter(TR[0], TR[1], label="Top Right")

BL = [dic["x_BL"], dic["y_BL"]]
plt.scatter(BL[0], BL[1], label="Bottom Left")

BR = [dic["x_BR"], dic["y_BR"]]
plt.scatter(BR[0], BR[1], label="Bottom Right")

points_2D_LARD = torch.tensor([TL, TR, BL, BR], dtype=torch.float64).unsqueeze(0)
points_2D_LARD_np = points_2D_LARD.squeeze(0).numpy()
print("Points 2D:\n", points_2D_LARD)

plt.legend()
plt.show()

### 2. Compute the rotation matrix
- from the yaw pitch and rool  
- given for this image, the one we want to re-estimate  

In [ ]:
from poseidon.numpy import generate_rotation_matrix_with_angles

yaw, pitch, roll = dic["yaw"], dic["pitch"], dic["roll"]
print("yaw,pitch,roll :", yaw, pitch, roll, "\n")

R = torch.tensor(
    generate_rotation_matrix_with_angles(yaw, pitch, roll), dtype=torch.float64
).unsqueeze(0)

print("Rotation matrix (R): \n", R, "\n", R.shape)

### 3. Recover the position matrix

In [ ]:
C = torch.tensor(
    [[dic["lon_cam"]], [dic["lat_cam"]], [dic["alt_cam"]]], dtype=torch.float64
).unsqueeze(0)

print("Camera position (C) : \n", C, "\n", C.shape)

### 4. Create the intraseca matrix of the camera

In [ ]:
# Intraseca
A = torch.tensor([[60, 0, 0], [0, 60, 0], [0, 0, 1]], dtype=torch.float64).unsqueeze(0)

print("Camera intrinsic parameters (A): \n", A, "\n", A.shape)

### 5. Import the 3D coordinates of the corners 

In [ ]:
# Load the runway points data
with open("runway_points.json", "r") as f:
    data = json.load(f)

# Extract the 3D coordinates of the corners
positions_lat = []
for key in ["B", "A", "C", "D"]:
    coordinate = data["CYEG"]["20"][key]["coordinate"]
    positions_lat.append([coordinate["longitude"], coordinate["latitude"], coordinate["altitude"]])

# Convert to numpy array and torch tensor
points_3D_np = np.array(positions_lat)
points_3D = torch.tensor(points_3D_np, dtype=torch.float64).unsqueeze(0)

print("3D points (torch):\n", points_3D, "\n", points_3D.shape)

Plot the four 3D points : 

In [ ]:
# Create of the 3D figure
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection="3d")

# Plot the camera center
ax.scatter(C[:, 0].squeeze(), C[:, 1].squeeze(), C[:, 2].squeeze(), color="orange")

# Plot the features points
for i in range(4):
    Pi = points_3D[:, i].squeeze()
    print(f"P{i+1}:", Pi)
    ax.scatter(*Pi, color="black")
    ax.text(*Pi, f"$P_{i+1}$")
    ax.plot(
        [C[:, 0].squeeze(), Pi[0]], [C[:, 1].squeeze(), Pi[1]], [C[:, 2].squeeze(), Pi[2]], "k--"
    )

# Set axis labels and limits
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")

# Plot the figure
ax.set_title("The three 3D Points use for P3P")
ax.view_init(elev=15, azim=30)  # élévation de 60°, azimut de 30°
ax.grid(True)
plt.tight_layout()
plt.show()

### 6. Generate the 2D points for the P3P problem

In [ ]:
from poseidon.torch import generate_synthetic_2D3Dpoints

points_2D = generate_synthetic_2D3Dpoints(R, C, A, points_3D)
points_2D_np = points_2D.squeeze(0).numpy()

print("Computed 2D points : \n", points_2D, points_2D.shape)
print("\n LARD dataset 2D points : \n", points_2D_LARD)

### 7. Compute the features vectors 

In [ ]:
from poseidon.torch import get_feature_vectors

features_vectors = get_feature_vectors(points_2D[:, :3], A)
print("Features vectors:\n", features_vectors, features_vectors.shape)
# Warning: Only the first 3 points are considered

In [ ]:
from poseidon.torch import compute_features_vectors, projection_all_point3D_to2D

old_points_2D = projection_all_point3D_to2D(points_3D, C, R, A)
old_features_vectors = compute_features_vectors(points_3D, C, R)
print("Old features vectors:\n", old_features_vectors, "\n", old_features_vectors.shape)

### Test with open cv : no solution found

In [ ]:
from poseidon.numpy import solve_reformat_p3p_solutions

# Convert intraseca matrix A to numpy array
A_np = A.squeeze(0).numpy()

# Apply the P3P algorithm from openCV
solutions_opencv = solve_reformat_p3p_solutions(points_3D_np[:3, :], points_2D_LARD_np[:3, :], A_np)
print("Solutions from OpenCV P3P:\n", solutions_opencv)

### Apply the P3P algorithm in torch

In [ ]:
from poseidon.torch import P3P

solutions_torch = P3P(points_3D, old_features_vectors)
print("Solutions from Poseidon P3P:\n", solutions_torch)

### Find the best solution 

In [ ]:
from poseidon.torch import find_best_solution_P3P_batch

R_solutions, C_solutions, error = find_best_solution_P3P_batch(
    solutions_torch, points_2D, points_3D, A
)
print("Rotation solutions (R):\n", R_solutions)
print("R", R)
print("Camera position solutions (C):\n", C_solutions)
print("C", C)
print("Error for each solution:\n", error)